# Handling duplicate, missing, or invalid data
## 중복, 결측, 유효하지 않은 데이터 다루기

## About the data
In this notebook, we will using daily weather data that was taken from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2) and altered to introduce many common problems faced when working with data. 

*Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for "NCEI weather API" to find the updated one.*

## Background on the data

Data meanings:
- `PRCP`: precipitation in millimeters / 밀리미터 단위의 강수량
- `SNOW`: snowfall in millimeters / 밀리미터 단위의 강설량
- `SNWD`: snow depth in millimeters / 밀리미터 단위의 적설량
- `TMAX`: maximum daily temperature in Celsius / 일일 최고 섭씨 온도
- `TMIN`: minimum daily temperature in Celsius / 일일 최저 섭씨 온도
- `TOBS`: temperature at time of observation in Celsius / 관측 시의 섭씨 온도
- `WESF`: water equivalent of snow in millimeters / 밀리미터 단위의 눈에 해당하는 물의 양.

Some important facts to get our bearings:
- According to the National Weather Service, the coldest temperature ever recorded in Central Park was -15°F (-26.1°C) on February 9, 1934: [source](https://www.weather.gov/media/okx/Climate/CentralPark/extremes.pdf) 
- The temperature of the Sun's photosphere is approximately 5,505°C: [source](https://en.wikipedia.org/wiki/Sun)

## Setup

In [2]:
import pandas as pd

df = pd.read_csv('data/dirty_data.csv')

## Finding problematic data
### 데이터셋에서 문제가 있는 데이터 찾기

**1. row 행들 살펴보기**

In [3]:
df.head() 

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False


-> 
* station 필드에는 ?와 관측소 ID가 섞여 있다.
* SNWD 적설량에는 음의 무한대값(-inf)가 있다.
* TMAX 최고 섭씨 온도에는 너무 큰 값이 있다.(ex. 5505.0)
* inclement_weather 필드와 몇몇 열에는 많은 NaN 값이 있다.

**2. describe() 요약통계를 사용하여 결측값이 있는지 확인하고, 잠재적인 문제가 있는지 살펴볼 수 있다.**

In [4]:
df.describe()

/Users/kimminyoung/opt/anaconda3/envs/danal/lib/python3.8/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF
count,765.000000,577.000000,577.0,765.000000,765.000000,398.000000,11.000000
mean,5.360392,4.202773,NaN,2649.175294,-15.914379,8.632161,16.290909
std,10.002138,25.086077,NaN,2744.156281,24.242849,9.815054,9.489832
min,0.000000,0.000000,-inf,-11.700000,-40.000000,-16.100000,1.800000
25%,0.000000,0.000000,NaN,13.300000,-40.000000,0.150000,8.600000
50%,0.000000,0.000000,NaN,32.800000,-11.100000,8.300000,19.300000
75%,5.800000,0.000000,NaN,5505.000000,6.700000,18.300000,24.900000
max,61.700000,229.000000,inf,5505.000000,23.900000,26.100000,28.700000


In [5]:
df.SNWD.unique()

array([-inf,  inf,  nan])

In [6]:
df.SNWD.value_counts()

-inf    553
 inf     24
Name: SNWD, dtype: int64

->
* SNWD 열은 쓸모없는 것으로 보인다. NaN값과 inf, -inf로 이루어져 있다. 
* TMAX 열은 신뢰할 수 없다. 너무 높은 온도 5505도가 있을 수 없다.

**3. `info()` 메서드를 사용하면 결측값이 있는지 알 수 있으며 열의 데이터 유형이 예상한 유형인지 확인할 수 있다.**

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               765 non-null    object 
 1   station            765 non-null    object 
 2   PRCP               765 non-null    float64
 3   SNOW               577 non-null    float64
 4   SNWD               577 non-null    float64
 5   TMAX               765 non-null    float64
 6   TMIN               765 non-null    float64
 7   TOBS               398 non-null    float64
 8   WESF               11 non-null     float64
 9   inclement_weather  408 non-null    object 
dtypes: float64(7), object(3)
memory usage: 59.9+ KB


info 결과를 보면총 765개의 row가 있지만, 5개의 열은 765보다 작다.

**4. `isna()`/`isnull()` 메서드를 사용해 null값이 들어간 행이 모두 포함되도록 데이터 프레임 생성.**

In [8]:
contain_nulls = df[
    df.SNOW.isna() | df.SNWD.isna() | df.TOBS.isna()
    | df.WESF.isna() | df.inclement_weather.isna()
]
contain_nulls.shape[0]

765

In [9]:
contain_nulls.head(10)

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
6,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
7,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
9,2018-01-05T00:00:00,?,0.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN


** `NaN` 값을 아래와 같은 방식으로 확인할 수 없다는 것을 주의해야한다.

데이터의 `NaN`값은 어떠한 값과도 같지 않기 때문에 **isna(), isnull()** 메서드를 사용해야한다.

In [10]:
df[df.inclement_weather == 'NaN'].shape[0]

0

In [11]:
import numpy as np
df[df.inclement_weather == np.nan].shape[0]

0

In [12]:
df[df.inclement_weather.isna()].shape[0]

357

`-inf`/`inf`는 실제로 `-np.inf`/`np.inf`이기 때문에,

`-np.inf`/`np.inf`로 음의무한대와 양의 무한대의 개수를 세어야 한다.

In [13]:
df[df.SNWD.isin([-np.inf, np.inf])].shape[0]

577

In [14]:
# 딕셔너리 컴프리헨션 사용
def get_inf_count(df):
    """데이터 프레임에서 컬럼 별로 inf와 -inf를 세는 함수"""
    return {
        col: df[df[col].isin([np.inf, -np.inf])].shape[0] for col in df.columns
    }

get_inf_count(df)

{'date': 0,
 'station': 0,
 'PRCP': 0,
 'SNOW': 0,
 'SNWD': 577,
 'TMAX': 0,
 'TMIN': 0,
 'TOBS': 0,
 'WESF': 0,
 'inclement_weather': 0}

적설량의 무한대 값을 어떻게 처리할 것인지 결정하기 전에 우리는 강설량에 대한 요약통계를 살펴봐야 한다. 강설량은 적설량을 결정하는데에 많은 비중을 차지한다.

inf와 -inf 각각으로 시리즈의 요약통계를 만들어 DataFrame 구조에 넣는다.

In [15]:
pd.DataFrame({
    'np.inf Snow Depth': df[df.SNWD == np.inf].SNOW.describe(),
    '-np.inf Snow Depth': df[df.SNWD == -np.inf].SNOW.describe()
}).T #데이터를 쉽게 보기 위해 전치

,count,mean,std,min,25%,50%,75%,max
np.inf Snow Depth,24.0,101.041667,74.498018,13.0,25.0,120.5,152.0,229.0
-np.inf Snow Depth,553.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


적설량은 눈이 내리지 않았을 때, 음의 무한대로 기록됐다.

고정된 날짜 범위에 대해 작업해야 하는경우 눈이 오지 않았다면 적설량은 0이나 NaN으로 처리하는 것이 좋겠다.

하지만, 양의 무한대는 어떻게 봐야할지 모르겠다. 그대로 놔두거나 사용하지 않는 것이 좋겠다.

**5. `date` and `station` 컬럼에 대해 요약통계를 생성해보면,**

* station 필드에는 총 2개의 값이 있는데 top값으로 나와있는 GHCND:USC00280907	와 앞에서 살펴봤듯이 ?인 것으로 알 수 있다.

* date필드에서는 freq값을 보면 어떤 날짜가 8번이나 들어가 있는 것으로 보인다. 우리는 unique값을 통해 날짜는 총 324날이 있는 것을 알 수 있다. 관측소 ID는 2개의 값이 들어가 있으므로 매일 2개의 항목이 있다고 가정해보면 648개의 데이터만 있어도 된다.

* 총 데이터의 개수는 765개 이므로 우리는 아직 무언가를 놓치고 있다.

In [16]:
df.describe(include='object') #문자열 열에 대한 통계적 요약 정보

,date,station,inclement_weather
count,765,765,408
unique,324,2,2
top,2018-07-05T00:00:00,GHCND:USC00280907,False
freq,8,398,384


`duplicated()` 메서드의 결과를 부울마스크로 사용하면 중복된 행을 찾을 수 있다.

In [17]:
df[df.duplicated()].shape[0]

284

`keep` 인수의 default 값은 `'first'` 로 중복되는 행들 중 첫번째 행 외의 추가 행만 가져온다.

아래에서는 keep = False 로 설정해 한 번 이상 나타나는 모든 행을 가져온다.

In [18]:
df[df.duplicated(keep=False)].shape[0]

482

사용할 컬럼들만 따로 볼 수도 있다.

In [19]:
df[df.duplicated(['date', 'station'])].shape[0]

284

In [20]:
df[df.duplicated()].head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
6,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True


->
* 위의 5개의 행만 봤을 때, 어떤 행은 적어도 세 번 반복된다는 것을 알 수 있다. (첫 행은 포함되지 않으므로)

* 이는 1행과 2행의 데이터에서 다른 대응값을 갖는다는 것을 뜻한다.

## Mitigating Issues
### 문제 완화하기

### Handling duplicated data
#### 중복된 행 처리하기

우리에게 중요한 것은 날씨 데이터이기 때문에 2개의 관측소가 모두 뉴욕시에 있다는 것을 알고 있으므로 `station` 필드를 삭제할 수도 있다. 그렇지만, 두 관측소가 서로 다른 데이터를 수집했을 수도 있다. 고로, 우리는 중복된 행을 제거할 때 나타날 수 있는 영향을 잘 평가하고 진행해야 한다.

중복된 행을 제거했다고 치고, date열을 사용해 중복행을 제거하고, `?`값이 아닌 관측소의 데이터를 남겨준다면, `?` 관측소에서만 WESF 측정을 기록했기 때문에 모든 `WESF` 기록은 사라진다.

In [21]:
df[df.WESF.notna()].station.unique() #notna():null값이 아닌 것만 부울값으로 마스크에 넣기 

array(['?'], dtype=object)

중복된 행들이 분석에 영향을 주지 않기로 했다면 `drop_duplicates()` 메서드를 사용해 **중복된 행을 제거**한다.

In [24]:
# 1. date 열에 대해 데이터 유형을 변환한다. object->datetime
df.date = pd.to_datetime(df.date)

# 2. 날짜를 기준으로 중복된 행들을 제거하고, WESF 열을 시리즈로 저장한다.
station_qm_wesf = df[df.station == '?']\
    .drop_duplicates('date').set_index('date').WESF

# 3. ID가 없는 관측소가 마지막에 위치하도록 station 열을 기준으로 
# DataFrame을 내림차순으로 정렬한다.
df.sort_values('station', ascending=False, inplace=True)

# 4. 날짜를 기준으로 station열에서 ID가 있는 첫번째 행은 놔두고 나머지 중복된 행을 제거한다.
df_deduped = df.drop_duplicates('date')

# 5. station열을 제거하고 date열을 인덱스로 설정
df_deduped = df_deduped.drop(columns='station').set_index('date').sort_index()

# 6. combine_first 메서드를 사용해 WESF 열을 업데이트해 값이 null이 아닌 첫번째 항목으로 병합
df_deduped = df_deduped.assign(
    WESF=lambda x: x.WESF.combine_first(station_qm_wesf)
)

df_deduped.shape

(324, 8)

In [26]:
df_deduped

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,NaN,True
...,...,...,...,...,...,...,...,...
2018-12-27,0.0,0.0,-inf,5.6,-2.2,-1.1,NaN,False
2018-12-28,11.7,0.0,-inf,6.1,-1.7,5.0,NaN,False
2018-12-29,21.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN


`combine_first()` 메소드를 사용해 WESF 열을 업데이트해 값이 null이 아닌 첫번째 항목으로 병합한다. 즉 2개의 관측소에 데이터가 있다면 ID가 있는 관측소의 값을 가져오고 해당 관측소가 null이면 ID가 없는 관측소의 값을 가져온다. df_deduped와 station_qm_wesf 모두 날짜를 인덱스로 사용하기 때문에 값은 적절한 날짜에 적절하게 대응된다:

| station GHCND:USC00280907 | station ? | result of `combine_first()` |
| :---: | :---: | :---: |
| 1 | 17 | 1 |
| 1 | `NaN` | 1 |
| `NaN` | 17 | 17 |
| `NaN` | `NaN` | `NaN` |


In [25]:
df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,NaN,True


### Dealing with nulls
#### null 데이터 처리

null 데이터를 삭제하거나 임의의 값으로 바꾸거나 주변 데이터로 대치할 수 있다. 
이러한 방법들은 모두 결과가 다르며 데이터에 미칠 영향을 고려해야 한다.

`dropna()` 메서드를 사용하면 null 데이터가 있는 모든 행을 삭제할 수 있다.

In [26]:
df_deduped.dropna().shape

(4, 8)

`how='all'` how인수를 사용하면 모든 열이 null인 경우에만 행을 삭제할 수 있으므로 아무것도 제거 되지 않았다.

In [27]:
df_deduped.dropna(how='all').shape

(324, 8)

삭제할 대상을 결정하고자 `subset`인수를 사용해 열의 부분집합을 사용할 수 있다.

inclement_weather, SNOW, SNWD **세 개의 열이 모두 null값일 때만** 삭제한다.

In [31]:
df_deduped.dropna(
    how='all', subset=['inclement_weather', 'SNOW', 'SNWD']
).shape

(293, 8)

적어도 한 열의 데이터가 75퍼 이상일 때만 열을 제거할 수도 있다. 임계값 thresh 지정.

In [27]:
df_deduped.dropna(axis='columns', thresh=df_deduped.shape[0] * .75).columns

Index(['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'inclement_weather'], dtype='object')

우리는 `fillna()`메서드를 이용해 "모든 행, WESF열만 채우기"와 같이 선택해 null값을 채워줄 수 있다.

In [28]:
df_deduped.loc[:,'WESF'].fillna(0, inplace=True)
df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


기온 데이터와 관련해 남아 있는 문제를 해결해본다면, TMAX값이 5505.0와 -40일 수는 없으므로 해당 값을 `NaN`값으로 대체한다.

In [29]:
df_deduped = df_deduped.assign(
    TMAX=lambda x: x.TMAX.replace(5505, np.nan),
    TMIN=lambda x: x.TMIN.replace(-40, np.nan),
)

또한 기온이 날마다 크게 변하지 않을 것이기에 `ffill`과 `bfill`을 이용해 채워본다.
- `'ffill'` 순방향 채우기 (현재 줄보다 앞의 값에서 대체할 값 가져오기)
- `'bfill'` 역방향 채우기
- `nearest` 가장 가까이에 있는 값으로 채우기

In [30]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(method='ffill'),
    TMIN=lambda x: x.TMIN.fillna(method='ffill')
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,-4.4,-13.9,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


SNWD 열의 null과 무한대 값을 처리하려면 `np.nan_to_num()` 함수를 사용해야 한다.
null값을 0으로 inf와 -inf를 매우 큰 양수 음수 값으로 변환해야 머신러닝 모델에 학습시킬 수 있다.

In [31]:
df_deduped.assign(
    SNWD=lambda x: np.nan_to_num(x.SNWD)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-1.797693e+308,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-1.797693e+308,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-1.797693e+308,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,1.797693e+308,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,1.797693e+308,-4.4,-13.9,-13.9,0.0,True


 - -np.inf는 해당 날짜에 눈이 내리지 않은 것이므로 SNWD값을 0으로 설정
 - np.inf는 어떻게 할지 미정.


작업하고 있는 데이터에 따라 `np.nan_to_num()` 메서드 대신 `clip()`메서드를 이용할 수도 있다. `clip()`메서드는 특정 최소/최대 임계값으로 값을 제한할 수 있다.
SNWD 값 적설량은 음의 값이 될 수 없으므로 clip() 함수로 하한을 0으로 설정한다.
상한이 어떻게 작동하는지 알아보고자 강설량을 추정값으로 사용.

- clip(하한값, 상한값)

In [32]:
df_deduped.assign(
    SNWD=lambda x: x.SNWD.clip(0, x.SNOW)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,0.0,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,0.0,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,229.0,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,127.0,-4.4,-13.9,-13.9,0.0,True


- **대치** : 요약통계나 다른 관측값의 데이터를 사용해 데이터에서 파생된 새로운 값으로 대체하는 것

대치작업을 `fillna()`메서드를 이용해 진행한다. 
 - TMAX의 null값과, TMIN의 null값을 각각의 중위수로 채운다. 
 - TOBS를 TMIN와 TMAX의 평균으로 채운다.

In [33]:
df_deduped.assign(
    TMAX=lambda x: x.TMAX.fillna(x.TMAX.median()),
    TMIN=lambda x: x.TMIN.fillna(x.TMIN.median()),
    # average of TMAX and TMIN
    TOBS=lambda x: x.TOBS.fillna((x.TMAX + x.TMIN) / 2)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,14.4,5.6,10.0,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,14.4,5.6,10.0,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


모든 열에 대해 같은 계산을 하려면 assign() 대신 `apply()` 메서드를 사용해야 한다.
- assign()은 각 열에 따로따로 계산식을 넣어주어야 하고
- apply()는 모든 열에 똑같이 계산식 적용됨.

apply() 가 각 열에 대해 같은 계산을 하도록 코드를 중복해서 작성하지 않아도 되기 때문이다.
 - 모든 결측값을 7일 이동 중위수로 채우고 추가 null값이 만들어지지 않도록 계산에 필요한 기간을 0으로 설정.
 -`rolling()`: 

In [35]:
df_deduped.apply(
  #최소 기간(계산에 필요한 기간)을 0으로 설정해서 항상 결과를 얻도록 함.
    lambda x: x.fillna(x.rolling(7, min_periods=0).median())
).head(10)

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.30,-16.1,-12.20,0.0,False
2018-01-03,0.0,0.0,-inf,-4.40,-13.9,-13.30,0.0,False
2018-01-04,20.6,229.0,inf,-6.35,-15.0,-12.75,19.3,True
2018-01-05,14.2,127.0,inf,-4.40,-13.9,-13.90,0.0,True
2018-01-06,0.0,0.0,-inf,-10.00,-15.6,-15.00,0.0,False
2018-01-07,0.0,0.0,-inf,-11.70,-17.2,-16.10,0.0,False
2018-01-08,0.0,0.0,-inf,-7.80,-16.7,-8.30,0.0,False
2018-01-10,0.0,0.0,-inf,5.00,-7.8,-7.80,0.0,False


결측 데이터를 대치하는 또 다른 방법은 pandas가 `interpolate()` 메서드로 대치할 값을 계산하도록 하는 것.
기본적으로 `intetplate()` 메서드는 모든 행이 일정한 간격을 가진다는 가정하에 선형보간법을 사용한다.
일부 날짜의 데이터가 없는 일별 데이터이므로 먼저 재인덱싱을 해야 한다.

-> 이전에 없었던 1월 9일 데이터가 생성되는 효과(8일과 10일의 평균)

In [36]:
df_deduped\
    .reindex(pd.date_range('2018-01-01', '2018-12-31', freq='D'))\
    .apply(lambda x: x.interpolate())\
    .head(10)

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.10,-12.20,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.90,-13.30,0.0,False
2018-01-04,20.6,229.0,inf,-4.4,-13.90,-13.60,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.90,-13.90,0.0,True
2018-01-06,0.0,0.0,-inf,-10.0,-15.60,-15.00,0.0,False
2018-01-07,0.0,0.0,-inf,-11.7,-17.20,-16.10,0.0,False
2018-01-08,0.0,0.0,-inf,-7.8,-16.70,-8.30,0.0,False
2018-01-09,0.0,0.0,-inf,-1.4,-12.25,-8.05,0.0,NaN
2018-01-10,0.0,0.0,-inf,5.0,-7.80,-7.80,0.0,False


<hr>

<div style="overflow: hidden; margin-bottom: 10px;">
    <div style="float: left;">
         <a href="./4-reshaping_data.ipynb">
            <button>&#8592; Previous Notebook</button>
        </a>
    </div>
    <div style="float: right;">
        <a href="../../solutions/ch_03/solutions.ipynb">
            <button>Solutions</button>
        </a>
        <a href="../ch_04/1-querying_and_merging.ipynb">
            <button>Chapter 4 &#8594;</button>
        </a>
    </div>
</div>
<hr>